In [1]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

import re
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

from matplotlib import pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BILAL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import sys
sys. version

'3.10.9 | packaged by Anaconda, Inc. | (main, Mar  1 2023, 18:18:15) [MSC v.1916 64 bit (AMD64)]'

In [3]:
icd_categories_description = pd.read_csv('categories.csv')

icd_categories_description

,icd_code,description
0,A00,Cholera
1,A01,Typhoid and paratyphoid fevers
2,A010,Typhoid fever
3,A011,Paratyphoid fever A
4,A012,Paratyphoid fever B
...,...,...
24623,Z99,"Dependence on enabling machines and devices, n..."
24624,Z991,Dependence on respirator
24625,Z992,Dependence on renal dialysis
24626,Z993,Dependence on wheelchair


In [4]:
# Define the reformat function
def reformat(code):
    code = ''.join(code.split('.'))
    code = code[:3] + '.'
    return code

In [5]:
icd_categories_description = icd_categories_description[icd_categories_description['icd_code'].astype(str).apply(len) <= 3]

icd_categories_description['icd_code'] = icd_categories_description['icd_code'].apply(lambda x: reformat(x))
icd_categories_description.reset_index(drop = True)

C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\242373274.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  icd_categories_description['icd_code'] = icd_categories_description['icd_code'].apply(lambda x: reformat(x))


,icd_code,description
0,A00.,Cholera
1,A01.,Typhoid and paratyphoid fevers
2,A02.,Other salmonella infections
3,A03.,Shigellosis
4,A04.,Other bacterial intestinal infections
...,...,...
1905,Z95.,Presence of cardiac and vascular implants and ...
1906,Z96.,Presence of other functional implants
1907,Z97.,Presence of other devices
1908,Z98.,Other postprocedural states


In [6]:
icd_categories_description.index = icd_categories_description['icd_code']
icd_categories_description

,icd_code,description
icd_code,,
A00.,A00.,Cholera
A01.,A01.,Typhoid and paratyphoid fevers
A02.,A02.,Other salmonella infections
A03.,A03.,Shigellosis
A04.,A04.,Other bacterial intestinal infections
...,...,...
Z95.,Z95.,Presence of cardiac and vascular implants and ...
Z96.,Z96.,Presence of other functional implants
Z97.,Z97.,Presence of other devices


In [8]:
from transformers import AutoTokenizer, AutoModel
import torch

# Mean Pooling - Take attention mask into account for correct averaging
def meanpooling(output, mask):
    embeddings = output[0] # First element of model_output contains all token embeddings
    mask = mask.unsqueeze(-1).expand(embeddings.size()).float()
    return torch.sum(embeddings * mask, 1) / torch.clamp(mask.sum(1), min=1e-9)


In [9]:
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained("neuml/pubmedbert-base-embeddings")
model = AutoModel.from_pretrained("neuml/pubmedbert-base-embeddings")

def create_description_embeddings(description):


    # Tokenize sentences
    inputs = tokenizer(description, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        output = model(**inputs)

    # Perform pooling. In this case, mean pooling.
    embeddings = meanpooling(output, inputs['attention_mask'])
    
    return embeddings

In [51]:
icd_categories_description['embeddings'] = icd_categories_description['description'].apply(create_description_embeddings)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
C:\Users\BILAL\AppData\Local\Temp\ipykernel_21812\1376811980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  icd_categories_description['embeddings'] = icd_categories_description['description'].apply(create_description_embeddings)


In [54]:
icd_categories_description

,icd_code,description,embeddings
icd_code,,,
A00.,A00.,Cholera,"[[tensor(-0.2244), tensor(-0.3841), tensor(-0...."
A01.,A01.,Typhoid and paratyphoid fevers,"[[tensor(0.4584), tensor(0.1789), tensor(0.952..."
A02.,A02.,Other salmonella infections,"[[tensor(-0.9934), tensor(-0.1978), tensor(1.1..."
A03.,A03.,Shigellosis,"[[tensor(-0.3418), tensor(-0.5553), tensor(0.2..."
A04.,A04.,Other bacterial intestinal infections,"[[tensor(-1.2683), tensor(-0.1100), tensor(0.9..."
...,...,...,...
Z95.,Z95.,Presence of cardiac and vascular implants and ...,"[[tensor(-0.8509), tensor(1.0413), tensor(-0.7..."
Z96.,Z96.,Presence of other functional implants,"[[tensor(-0.8399), tensor(1.5485), tensor(0.44..."
Z97.,Z97.,Presence of other devices,"[[tensor(-0.7099), tensor(1.0142), tensor(-0.3..."


In [10]:
testing_notes_with_smallest_size = pd.read_csv('1000_testing_notes_with_smallest_size.csv')

In [11]:
testing_notes_with_smallest_size.drop(columns = ['Unnamed: 0'], inplace = True)

In [12]:
icd_codes_with_smallest_size_notes = pd.read_csv('icd_codes_for_1000_testing_notes_with_smallest_size.csv')

In [13]:
icd_codes_with_smallest_size_notes.drop(columns = ['Unnamed: 0'], inplace = True)

In [14]:
final_notes_with_icd_codes = pd.merge(testing_notes_with_smallest_size, icd_codes_with_smallest_size_notes, on='hadm_id', how='left')
final_notes_with_icd_codes

,hadm_id,text,text_length,icd_code
0,20019282,\nName: ___ Unit No: ___\n...,2480,S82.
1,20019282,\nName: ___ Unit No: ___\n...,2480,Z72.
2,20019282,\nName: ___ Unit No: ___\n...,2480,W10.
3,20019282,\nName: ___ Unit No: ___\n...,2480,Y92.
4,20022343,\nName: ___ Unit No: _...,3001,O60.
...,...,...,...,...
8580,29990058,\nName: ___ Unit No: ___\n...,3051,Z30.
8581,29990058,\nName: ___ Unit No: ___\n...,3051,Z23.
8582,29990058,\nName: ___ Unit No: ___\n...,3051,Z86.
8583,29990058,\nName: ___ Unit No: ___\n...,3051,K21.


In [15]:
medical_stopwords = stopwords.words("english")


medical_stopwords.extend(['speaking', 'none', 'time', 'flush'])

In [16]:
def process_clinical_note(clinical_note):
    # Define the sections to remove
    sections_to_remove = [
        "Name:",
        "Unit No:",
        "Admission Date:",
        "Discharge Date:",
        "Date of Birth:",
        "Sex:",
        "Service:",
        "Allergies:",
        "Attending:",
        "Past Medical History:",
        "Social History:",
        "Family History:",
        "Vitals:",
        "Pertinent Results:",
        "Medications on Admission:",
        "Discharge Medications:",
        "Discharge Disposition:",
        "Discharge Condition:",
        "Discharge Instructions:",
        "Followup Instructions:"
    ]

    # Split the clinical note into lines
    lines = clinical_note.split('\n')

    # Initialize the processed note
    processed_note = []

    # Flag to exclude lines within unwanted sections
    exclude_section = False

    # Iterate through the lines and filter unwanted sections
    for line in lines:
        if any(section in line for section in sections_to_remove):
            exclude_section = True
        elif line.strip() == "":
            # Empty lines separate sections, so reset the flag
            exclude_section = False

        if not exclude_section:
            processed_note.append(line)

    # Join the lines to create the final note
    final_note = '\n '.join(processed_note)
    
    sections_to_remove = [
        r'chief complaint',
        r'history of present illness',
        r'Major Surgical or Invasive Procedure',
        r'physical exam',
        r'brief hospital course',
        r'Discharge',
        
        r'completed by',
    ]
    
    for pattern in sections_to_remove:
        final_note = re.sub(pattern, '', final_note, flags=re.IGNORECASE)

    # Define patterns to identify negations
    negation_patterns = [
        r'no\s+\w+',
        r'not\s+\w+',
        r'did\s+not\s+have\s+\w+'
    ]
    
    # Filter out sentences with negations
    sentences = [sentence for sentence in final_note.split('\n') if not any(re.search(pattern, sentence, re.IGNORECASE) for pattern in negation_patterns)]

    # Remove keys and special characters
    cleaned_note = re.sub(r'\w+:', '', '\n'.join(sentences), flags=re.IGNORECASE)  # Remove keys (case-insensitive)
    cleaned_note = re.sub(r'[^a-zA-Z\s]', '', cleaned_note)  # Remove special characters
    # Tokenize the note into sentences based on '\n'
    sentences = [sentence.strip() for sentence in cleaned_note.split('\n') if sentence.strip()]

    # Remove stop words and empty sentences
    sentences = [
        ' '.join(word for word in sentence.split() if word.lower() not in medical_stopwords)
        for sentence in sentences
    ]
    sentences = [item for item in sentences if item != '']

    return sentences

In [17]:
import json
import pinecone


from collections import Counter
import numpy as np
from functools import partial
import string
import datasets

import re
import matplotlib.pyplot as plt


In [18]:
## Storing API keys
pinecone_api_key = "24c6f012-a062-4e67-b4cd-b9ab9b555cd0"
pinecone_environment = "gcp-starter"


In [104]:
## Creating pinecone index

pinecone.init(api_key=pinecone_api_key, environment=pinecone_environment)
pinecone_index_name = "msproject"

## delete the index if it already exists
if pinecone_index_name in pinecone.list_indexes():
    pinecone.delete_index(pinecone_index_name)

# create the index
pinecone.create_index(name=pinecone_index_name, metric="cosine", shards=1, dimension=768)

In [19]:
# First we import pinecone. If you have not installed it, install it first by running pip install pinecone
pinecone.init(pinecone_api_key, environment=pinecone_environment)
pinecone_index_name = "msproject"
# connect to the newly created index
index = pinecone.Index(pinecone_index_name)

In [128]:
reference_embeddings = torch.stack([torch.tensor(embedding) for embedding in icd_categories_description['embeddings'].values])
reference_embeddings = reference_embeddings.squeeze()
reference_embeddings = np.array([embedding.numpy() for embedding in reference_embeddings])

C:\Users\BILAL\AppData\Local\Temp\ipykernel_21812\2667019115.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  reference_embeddings = torch.stack([torch.tensor(embedding) for embedding in icd_categories_description['embeddings'].values])


In [129]:
reference_embeddings.shape

(1910, 768)

In [145]:
reference_embeddings = torch.stack([torch.tensor(embedding) for embedding in icd_categories_description['embeddings'].values])
reference_embeddings = reference_embeddings.squeeze()
reference_embeddings = [embedding.tolist() for embedding in reference_embeddings]
shape = np.array(reference_embeddings).shape
shape

C:\Users\BILAL\AppData\Local\Temp\ipykernel_21812\2617819856.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  reference_embeddings = torch.stack([torch.tensor(embedding) for embedding in icd_categories_description['embeddings'].values])


(1910, 768)

In [140]:
icd_categories_description.description.tolist()

['Cholera',
 'Typhoid and paratyphoid fevers',
 'Other salmonella infections',
 'Shigellosis',
 'Other bacterial intestinal infections',
 'Other bacterial foodborne intoxications, not elsewhere classified',
 'Amebiasis',
 'Other protozoal intestinal diseases',
 'Viral and other specified intestinal infections',
 'Infectious gastroenteritis and colitis, unspecified',
 'Respiratory tuberculosis',
 'Tuberculosis of nervous system',
 'Tuberculosis of other organs',
 'Miliary tuberculosis',
 'Plague',
 'Tularemia',
 'Anthrax',
 'Brucellosis',
 'Glanders and melioidosis',
 'Rat-bite fevers',
 'Erysipeloid',
 'Leptospirosis',
 'Other zoonotic bacterial diseases, not elsewhere classified',
 "Leprosy [Hansen's disease]",
 'Infection due to other mycobacteria',
 'Listeriosis',
 'Tetanus neonatorum',
 'Obstetrical tetanus',
 'Other tetanus',
 'Diphtheria',
 'Whooping cough',
 'Scarlet fever',
 'Meningococcal infection',
 'Streptococcal sepsis',
 'Other sepsis',
 'Actinomycosis',
 'Nocardiosis',
 

In [142]:
meta = []
for i in range(len(icd_categories_description)):
    
    temp = {}
    temp['icd_10_code'] = icd_categories_description['icd_code'][i]
    temp['description'] = icd_categories_description['description'][i]
    meta.append(temp)
len(meta)

1910

In [134]:
meta[0]

{'icd_10_code': 'A00.', 'description': 'Cholera'}

In [148]:
batch_size = 100

## ids
ids = [str(i) for i in range(len(icd_categories_description))]

# create list of (id, vector, metadata) tuples to be upserted
to_upsert = list(zip(ids, reference_embeddings, meta))
for i in range(0, shape[0], batch_size):
    i_end = min(i+batch_size, shape[0])
    index.upsert(vectors=to_upsert[i:i_end])

In [156]:
df_output = pd.DataFrame()

In [98]:
sentences = ['Left ankle fracture',
 'ORIF left ankle',
 'yr old man fall stairs holding lumbar',
 'construction',
 'job resulting immediate L ankle pain ED ankle fracture',
 'surgery performed',
 'AOXO',
 'splint intact positive CSM',
 'Taken operating room underwent surgical fixation ankle',
 'well controlled Cleared go home without services',
 'left ankle fracture sp ORIF']

In [217]:
sentences = ['Stroke',
 'yr F w hx ESRD HD HTN DMII prior stroke',
 'w residual mild aphasia transferred OSH due',
 'new onset R sided weakness profound aphasia concerning',
 'stroke Exam consistent L MCA syndrome w pt seen',
 'severe expressive aphasia L gaze preference R',
 'sided weakness NIHSS OSH tPA given OSH',
 'NIHSS Etiology could potentially',
 'cardioembolic given recurrent stroke proximal vasculature',
 'appearing clear affected side hour head CT showed',
 'large L MCA',
 'expected entire territory infarct',
 'appeared relatively good distal reconstitution',
 'fetal PCA L appeared supplying MCA',
 'infarct quite large affecting',
 'cortical motor areas subcortical fibers would expect',
 'remain hemiplegic R exam developed',
 'global aphasia hemiplegia R Goals care',
 'discussed family made comfort measures',
 'passed away',
 'ESRD received HD renal consulted',
 'management Per family request discontinued',
 'goals shifted comfort measures',
 'yeast peritoneal fluid yeast grew peritoneal fluid',
 'culture started fluconazole ID',
 'consulted discontinued made CMO',
 'Stroke'
, '']

In [218]:
notes = create_description_embeddings(sentences)
notes_embeddings = [embedding.tolist() for embedding in notes]

In [215]:
sentences_list = []
icd_10_code_list = []
description_list = []
score_list = []

for i in range(0, len(notes_embeddings)):
    similar = index.query(notes_embeddings[i], top_k=1, include_metadata=True)
    
    sentences_list.append(sentences[i])
    
    if similar['matches']:
        match = similar['matches'][0]
        icd_10_code_list.append(match['metadata']['icd_10_code'])
        description_list.append(match['metadata']['description'])
        score_list.append(match['score'])
    else:
        icd_10_code_list.append(None)
        description_list.append(None)
        score_list.append(None)

# Create a DataFrame
df_output = pd.DataFrame({
    'Sentences': sentences_list,
    'ICD_10_Code': icd_10_code_list,
    'Description': description_list,
    'Score': score_list
})



In [216]:
df_output[df_output.Score >=0.0]

,Sentences,ICD_10_Code,Description,Score
0,Stroke,I63.,Cerebral infarction,0.692511
1,yr F w hx ESRD HD HTN DMII prior stroke,N19.,Unspecified kidney failure,0.461103
2,w residual mild aphasia transferred OSH due,F04.,Amnestic disorder due to known physiological c...,0.426580
3,new onset R sided weakness profound aphasia co...,G81.,Hemiplegia and hemiparesis,0.447988
4,stroke Exam consistent L MCA syndrome w pt seen,G45.,Transient cerebral ischemic attacks and relate...,0.405531
5,severe expressive aphasia L gaze preference R,F80.,Specific developmental disorders of speech and...,0.418429
6,sided weakness NIHSS OSH tPA given OSH,R57.,"Shock, not elsewhere classified",0.405377
7,NIHSS Etiology could potentially,G03.,Meningitis due to other and unspecified causes,0.474772
8,cardioembolic given recurrent stroke proximal ...,I66.,"Occlusion and stenosis of cerebral arteries, n...",0.586582
9,appearing clear affected side hour head CT showed,R51.,Headache,0.502359


In [191]:
final_notes_with_icd_codes[final_notes_with_icd_codes.hadm_id == 20053836]

,hadm_id,text,text_length,icd_code
25,20053836,\nName: ___ Unit No: ___\n ...,2491,I63.
26,20053836,\nName: ___ Unit No: ___\n ...,2491,N18.
27,20053836,\nName: ___ Unit No: ___\n ...,2491,J96.
28,20053836,\nName: ___ Unit No: ___\n ...,2491,J18.
29,20053836,\nName: ___ Unit No: ___\n ...,2491,B48.
30,20053836,\nName: ___ Unit No: ___\n ...,2491,E11.
31,20053836,\nName: ___ Unit No: ___\n ...,2491,D68.
32,20053836,\nName: ___ Unit No: ___\n ...,2491,E11.
33,20053836,\nName: ___ Unit No: ___\n ...,2491,I12.
34,20053836,\nName: ___ Unit No: ___\n ...,2491,T85.


In [20]:


def get_icd_scores(sentences, index):
    result_list = []

    # Create embeddings for sentences
    notes_embeddings = create_description_embeddings(sentences)
    notes_embeddings = [embedding.tolist() for embedding in notes_embeddings]

    for i in range(len(sentences)):
        similar = index.query(notes_embeddings[i], top_k=1, include_metadata=True)

        if similar['matches']:
            match = similar['matches'][0]
            icd_code = match['metadata']['icd_10_code']
            score = match['score']
            result_list.append({'icd_code': icd_code, 'score': score})

    return result_list

In [21]:
import logging

# Configure logging to print to console
logging.basicConfig(level=logging.INFO)

# Assuming you have 'testing_notes_with_smallest_size' DataFrame and 'index' available



In [22]:
def get_icd_scores(sentences, index, row_count):
    result_list = []
    
    if len(sentences) != 0:
        # Create embeddings for sentences
        notes_embeddings = create_description_embeddings(sentences)
        notes_embeddings = [embedding.tolist() for embedding in notes_embeddings]

        for i, sentence in enumerate(sentences):
            try:
                similar = index.query(notes_embeddings[i], top_k=1, include_metadata=True)

                if similar['matches']:
                    match = similar['matches'][0]
                    icd_code = match['metadata']['icd_10_code']
                    score = match['score']
                    result_list.append({'icd_code': icd_code, 'score': score})

            except Exception as e:
                logging.error(f"Error processing row {row_count}: {str(e)}")

        # Print progress for every row processed
        logging.info(f"Processed {row_count} rows")

        return result_list
    
    else:
        return None


In [23]:
final_notes_with_icd_codes

,hadm_id,text,text_length,icd_code
0,20019282,\nName: ___ Unit No: ___\n...,2480,S82.
1,20019282,\nName: ___ Unit No: ___\n...,2480,Z72.
2,20019282,\nName: ___ Unit No: ___\n...,2480,W10.
3,20019282,\nName: ___ Unit No: ___\n...,2480,Y92.
4,20022343,\nName: ___ Unit No: _...,3001,O60.
...,...,...,...,...
8580,29990058,\nName: ___ Unit No: ___\n...,3051,Z30.
8581,29990058,\nName: ___ Unit No: ___\n...,3051,Z23.
8582,29990058,\nName: ___ Unit No: ___\n...,3051,Z86.
8583,29990058,\nName: ___ Unit No: ___\n...,3051,K21.


In [24]:
testing_notes_with_smallest_size.iloc[571]

hadm_id                                                 25543633
text            \nName:  ___                  Unit No:   ___\...
text_length                                                 1417
Name: 571, dtype: object

In [26]:
testing_notes_with_smallest_size['sentences'] = testing_notes_with_smallest_size['text'].apply(process_clinical_note)
testing_notes_with_smallest_size

,hadm_id,text,text_length,sentences
0,20019282,\nName: ___ Unit No: ___\n...,2480,"[Left ankle fracture, ORIF left ankle, yr old ..."
1,20022343,\nName: ___ Unit No: _...,3001,"[Preterm labor, repeat low transverse cesarean..."
2,20025548,\nName: ___ Unit No: ___\n...,2961,"[contractions, SVD, Pt arrives co painful cont..."
3,20027416,\nName: ___ Unit No: ___\n...,1059,"[neck cancer, direct laryngoscopy robotic hemi..."
4,20028418,\nName: ___ Unit No: ___\n \...,1091,"[admission chemo, Pt tolerated desens without ..."
...,...,...,...,...
1010,29980814,\nName: ___ Unit No: _...,3329,"[Postpartum incision check, partial opening pa..."
1011,29980930,\nName: ___ Unit No: ___...,1094,"[elective chemotherapy, Pt presented elective ..."
1012,29987299,\nName: ___ Unit No: ___\...,2595,"[attach, BLOOD WBC RBC Hgb Hct, MCV MCH MCHC R..."
1013,29988390,\nName: ___ Unit No: ...,3013,"[acute cholecystitis, laparoscopic cholecystec..."


In [27]:
# Apply the function to the 'sentences' column and create a new 'predictions' column
testing_notes_with_smallest_size['predictions'] = testing_notes_with_smallest_size.apply(lambda row: get_icd_scores(row['sentences'], index, row.name), axis=1)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
INFO:root:Processed 0 rows
INFO:root:Processed 1 rows
INFO:root:Processed 2 rows
INFO:root:Processed 3 rows
INFO:root:Processed 4 rows
INFO:root:Processed 5 rows
INFO:root:Processed 6 rows
INFO:root:Processed 7 rows
INFO:root:Processed 8 rows
INFO:root:Processed 9 rows
INFO:root:Processed 10 rows
INFO:root:Processed 11 rows
ERROR:root:Error processing row 12: Failed to connect; did you specify the correct index name?
INFO:root:Processed 12 rows
INFO:root:Processed 13 rows
INFO:root:Processed 14 rows
INFO:root:Processed 15 rows
INFO:root:Processed 16 rows
INFO:root:Processed 17 rows
INFO:root:Processed 18 rows
INFO:root:Processed 19 rows
INFO:root:Processed 20 rows
INFO:root:Processed 21 rows
INFO:root:Processed 22 rows
INFO:root:Processed 23 rows
INFO:root:Processed 24 rows
INFO:root:Processed 25 rows
INFO:root:Processed 26 rows
INFO:root:Processe

INFO:root:Processed 279 rows
INFO:root:Processed 280 rows
INFO:root:Processed 281 rows
INFO:root:Processed 282 rows
INFO:root:Processed 283 rows
INFO:root:Processed 284 rows
INFO:root:Processed 285 rows
INFO:root:Processed 286 rows
INFO:root:Processed 287 rows
INFO:root:Processed 288 rows
INFO:root:Processed 289 rows
INFO:root:Processed 290 rows
INFO:root:Processed 291 rows
INFO:root:Processed 292 rows
INFO:root:Processed 293 rows
INFO:root:Processed 294 rows
INFO:root:Processed 295 rows
INFO:root:Processed 296 rows
INFO:root:Processed 297 rows
INFO:root:Processed 298 rows
INFO:root:Processed 299 rows
INFO:root:Processed 300 rows
INFO:root:Processed 301 rows
INFO:root:Processed 302 rows
INFO:root:Processed 303 rows
INFO:root:Processed 304 rows
INFO:root:Processed 305 rows
INFO:root:Processed 306 rows
INFO:root:Processed 307 rows
INFO:root:Processed 308 rows
INFO:root:Processed 309 rows
INFO:root:Processed 310 rows
INFO:root:Processed 311 rows
INFO:root:Processed 312 rows
INFO:root:Proc

INFO:root:Processed 562 rows
INFO:root:Processed 563 rows
INFO:root:Processed 564 rows
INFO:root:Processed 565 rows
INFO:root:Processed 566 rows
INFO:root:Processed 567 rows
INFO:root:Processed 568 rows
INFO:root:Processed 569 rows
INFO:root:Processed 570 rows
INFO:root:Processed 572 rows
INFO:root:Processed 573 rows
INFO:root:Processed 574 rows
INFO:root:Processed 575 rows
INFO:root:Processed 576 rows
INFO:root:Processed 577 rows
INFO:root:Processed 578 rows
INFO:root:Processed 579 rows
INFO:root:Processed 580 rows
INFO:root:Processed 581 rows
INFO:root:Processed 582 rows
INFO:root:Processed 583 rows
INFO:root:Processed 584 rows
INFO:root:Processed 585 rows
INFO:root:Processed 586 rows
INFO:root:Processed 587 rows
INFO:root:Processed 588 rows
INFO:root:Processed 589 rows
INFO:root:Processed 590 rows
INFO:root:Processed 591 rows
INFO:root:Processed 592 rows
INFO:root:Processed 593 rows
INFO:root:Processed 594 rows
INFO:root:Processed 595 rows
INFO:root:Processed 596 rows
INFO:root:Proc

INFO:root:Processed 846 rows
INFO:root:Processed 847 rows
INFO:root:Processed 848 rows
INFO:root:Processed 849 rows
INFO:root:Processed 850 rows
INFO:root:Processed 851 rows
INFO:root:Processed 852 rows
INFO:root:Processed 853 rows
INFO:root:Processed 854 rows
INFO:root:Processed 855 rows
INFO:root:Processed 856 rows
INFO:root:Processed 857 rows
INFO:root:Processed 858 rows
INFO:root:Processed 859 rows
INFO:root:Processed 860 rows
INFO:root:Processed 861 rows
INFO:root:Processed 862 rows
INFO:root:Processed 863 rows
INFO:root:Processed 864 rows
INFO:root:Processed 865 rows
INFO:root:Processed 866 rows
INFO:root:Processed 867 rows
INFO:root:Processed 868 rows
INFO:root:Processed 869 rows
INFO:root:Processed 870 rows
INFO:root:Processed 871 rows
INFO:root:Processed 872 rows
INFO:root:Processed 873 rows
INFO:root:Processed 874 rows
INFO:root:Processed 875 rows
INFO:root:Processed 876 rows
INFO:root:Processed 877 rows
INFO:root:Processed 878 rows
INFO:root:Processed 879 rows
INFO:root:Proc

In [28]:
testing_notes_with_smallest_size

,hadm_id,text,text_length,sentences,predictions
0,20019282,\nName: ___ Unit No: ___\n...,2480,"[Left ankle fracture, ORIF left ankle, yr old ...","[{'icd_code': 'S82.', 'score': 0.752511084}, {..."
1,20022343,\nName: ___ Unit No: _...,3001,"[Preterm labor, repeat low transverse cesarean...","[{'icd_code': 'O60.', 'score': 0.999484837}, {..."
2,20025548,\nName: ___ Unit No: ___\n...,2961,"[contractions, SVD, Pt arrives co painful cont...","[{'icd_code': 'P94.', 'score': 0.430006832}, {..."
3,20027416,\nName: ___ Unit No: ___\n...,1059,"[neck cancer, direct laryngoscopy robotic hemi...","[{'icd_code': 'C01.', 'score': 0.559164345}, {..."
4,20028418,\nName: ___ Unit No: ___\n \...,1091,"[admission chemo, Pt tolerated desens without ...","[{'icd_code': 'D70.', 'score': 0.455312133}, {..."
...,...,...,...,...,...
1010,29980814,\nName: ___ Unit No: _...,3329,"[Postpartum incision check, partial opening pa...","[{'icd_code': 'Z39.', 'score': 0.534763038}, {..."
1011,29980930,\nName: ___ Unit No: ___...,1094,"[elective chemotherapy, Pt presented elective ...","[{'icd_code': 'D70.', 'score': 0.460205525}, {..."
1012,29987299,\nName: ___ Unit No: ___\...,2595,"[attach, BLOOD WBC RBC Hgb Hct, MCV MCH MCHC R...","[{'icd_code': 'W12.', 'score': 0.485362917}, {..."
1013,29988390,\nName: ___ Unit No: ...,3013,"[acute cholecystitis, laparoscopic cholecystec...","[{'icd_code': 'K81.', 'score': 0.919016063}, {..."


In [35]:
testing_notes_with_smallest_size[testing_notes_with_smallest_size.predictions == None]

,hadm_id,text,text_length,sentences,predictions


In [36]:

# Create the resultant DataFrame
result_df = pd.DataFrame()

for index, row in testing_notes_with_smallest_size.iterrows():
    hadm_id = str(row['hadm_id'])
    predictions = row['predictions']

    # Dictionary to store the maximum score for each label within a row
    max_scores = {}
    
    if predictions:
        for prediction in predictions:
            label = prediction['icd_code']
            score = prediction['score']

            # Update the max score for the label if a higher score is found
            if label in max_scores:
                max_scores[label] = max(max_scores[label], score)
            else:
                max_scores[label] = score

        # Append the results to the DataFrame
        for label, max_score in max_scores.items():
            # Optional: Specify a threshold (e.g., 0.20) to filter predictions
            if max_score >= 0.50:
                result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)

# Optionally, you can sort the DataFrame by 'hadm_id' and 'Score' if needed
result_df.sort_values(by=['hadm_id', 'Score'], ascending=[True, False], inplace=True)


C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel

C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel

C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel

C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel

C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel

...

C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel

C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel

C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel

C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel

C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_6864\2420831353.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel

In [37]:
# Define the reformat function
def reformat(code):
    code = ''.join(code.split('.'))
    code = code[:3] + '.'
    return code

In [38]:
result_df['icd_10_predicted'] = result_df['icd_10_predicted'].apply(lambda x: reformat(x))
result_df

,hadm_id,icd_10_predicted,Score
0,20019282,S82.,0.752511
1,20019282,S97.,0.672783
2,20019282,W06.,0.624991
4,20019282,Y83.,0.624054
5,20019282,Z40.,0.530660
...,...,...,...
8818,29990058,O69.,0.604733
8823,29990058,R03.,0.596206
8816,29990058,H26.,0.552230
8822,29990058,Z08.,0.530658


In [39]:
result_df.dtypes

hadm_id              object
icd_10_predicted     object
Score               float64
dtype: object

In [152]:
unique_icd_codes = set(final_notes_with_icd_codes['icd_code'])
len(unique_icd_codes)

769

In [153]:
result_df

,hadm_id,icd_10_predicted,Score
0,20019282,S82.,0.752511
1,20019282,S97.,0.672783
2,20019282,W06.,0.624991
4,20019282,Y83.,0.624054
5,20019282,Z40.,0.530660
...,...,...,...
8818,29990058,O69.,0.604733
8823,29990058,R03.,0.596206
8816,29990058,H26.,0.552230
8822,29990058,Z08.,0.530658


In [154]:
# Step 3: Initialize TP, FP, FN, TN counts
counts = {icd_code: (0, 0, 0, 0) for icd_code in unique_icd_codes}


In [155]:
final_notes_with_icd_codes['hadm_id'] = final_notes_with_icd_codes['hadm_id'].astype(str)

In [156]:
i = 0
for icd_code in unique_icd_codes:
    i+=1
    if i%100 == 0:
        print(f'{i} codes are done!')
    
    true_icd_codes = final_notes_with_icd_codes[final_notes_with_icd_codes['icd_code'] == icd_code]['hadm_id'].values
    predicted_icd_codes = result_df[result_df['icd_10_predicted'] == icd_code]['hadm_id'].values

    TP = np.intersect1d(true_icd_codes, predicted_icd_codes)
    FP = np.setdiff1d(predicted_icd_codes, true_icd_codes)
    FN = np.setdiff1d(true_icd_codes, predicted_icd_codes)
    TN = len(np.union1d(predicted_icd_codes, true_icd_codes)) -len(TP) -len(FP) -len(FN)


    counts[icd_code] = (len(TP), len(FP), len(FN), TN)

100 codes are done!
200 codes are done!
300 codes are done!
400 codes are done!
500 codes are done!
600 codes are done!
700 codes are done!


In [157]:
counts

{'J98.': (0, 0, 12, 0),
 'J20.': (0, 3, 1, 0),
 'M35.': (0, 0, 5, 0),
 'I77.': (0, 3, 5, 0),
 'Y82.': (0, 0, 1, 0),
 'F02.': (0, 0, 10, 0),
 'K92.': (0, 0, 13, 0),
 'R20.': (0, 43, 8, 0),
 'X58.': (0, 1, 32, 0),
 'Z53.': (0, 9, 13, 0),
 'R43.': (0, 3, 1, 0),
 'E66.': (5, 7, 92, 0),
 'J04.': (0, 3, 1, 0),
 'D11.': (0, 6, 2, 0),
 'O71.': (0, 5, 8, 0),
 'I83.': (1, 18, 3, 0),
 'M34.': (0, 0, 2, 0),
 'M62.': (0, 0, 10, 0),
 'R14.': (0, 7, 1, 0),
 'Z37.': (32, 7, 54, 0),
 'Q74.': (0, 1, 1, 0),
 'Z00.': (1, 72, 5, 0),
 'M41.': (3, 2, 1, 0),
 'K40.': (2, 10, 0, 0),
 'G20.': (0, 1, 8, 0),
 'G72.': (0, 0, 1, 0),
 'R56.': (0, 0, 6, 0),
 'Z3A.': (33, 21, 160, 0),
 'Z09.': (0, 12, 1, 0),
 'E03.': (12, 4, 62, 0),
 'K80.': (10, 8, 5, 0),
 'N80.': (1, 15, 3, 0),
 'L02.': (3, 5, 9, 0),
 'G30.': (3, 0, 5, 0),
 'S42.': (2, 19, 8, 0),
 'D49.': (0, 0, 2, 0),
 'R12.': (1, 14, 0, 0),
 'R42.': (0, 41, 7, 0),
 'O89.': (0, 4, 1, 0),
 'S20.': (0, 5, 1, 0),
 'W34.': (0, 0, 2, 0),
 'J44.': (1, 1, 44, 0),
 'S70.':

In [158]:
# Calculate Precision and Recall for Each ICD-10 Code
precisions = {}
recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    precisions[icd_code] = precision
    recalls[icd_code] = recall


In [159]:
# Step 6: Calculate Macro Precision and Macro Recall
macro_precision = sum(precisions.values()) / len(precisions)
macro_recall = sum(recalls.values()) / len(recalls)

In [160]:
macro_precision

0.1511054393484266

In [161]:
macro_recall

0.16114504457202314

In [162]:
# Calculate Micro Precision and Recall
total_TP = sum([counts[icd_code][0] for icd_code in unique_icd_codes])
total_FP = sum([counts[icd_code][1] for icd_code in unique_icd_codes])
total_FN = sum([counts[icd_code][2] for icd_code in unique_icd_codes])

# Handle division by zero
micro_precision = total_TP / (total_TP + total_FP) if (total_TP + total_FP) != 0 else 0
micro_recall = total_TP / (total_TP + total_FN) if (total_TP + total_FN) != 0 else 0

print("Micro Precision:", micro_precision)
print("Micro Recall:", micro_recall)

Micro Precision: 0.16371077762619374
Micro Recall: 0.12104400453915017


In [163]:
# Calculate F1 Score for Each ICD-10 Code
f1_scores = {}
for icd_code in unique_icd_codes:
    precision = precisions[icd_code]
    recall = recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    f1_scores[icd_code] = f1_score


In [164]:
# Step 8: Calculate Macro F1 Score
macro_f1_score = sum(f1_scores.values()) / len(f1_scores)

macro_f1_score

0.11654075966517526

In [165]:
# Handle division by zero for precision and recall in Micro F1 Score
if micro_precision + micro_recall == 0:
    micro_f1_score = 0
else:
    micro_f1_score = 2 * (micro_precision * micro_recall) / (micro_precision + micro_recall)

print("Micro F1 Score:", micro_f1_score)

Micro F1 Score: 0.13918086263138818


In [166]:
# Calculate Precision and Recall for Each ICD-10 Code
precisions = {}
recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    precisions[icd_code] = precision
    recalls[icd_code] = recall

# Filter out zero precision values and calculate average precision
average_precision = np.mean([precision for precision in precisions.values() if precision > 0])

In [167]:
average_precision

0.41205703141468075

In [168]:
average_recall = np.mean([recall for recall in recalls.values() if recall > 0])

In [169]:
average_recall

0.4394345364393111

In [170]:
# Calculate F1 Score for Each ICD-10 Code
f1_scores = {}
for icd_code in unique_icd_codes:
    precision = precisions[icd_code]
    recall = recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    f1_scores[icd_code] = f1_score


In [171]:
average_f1 = np.mean([f1_score for f1_score in f1_scores.values() if f1_score > 0])

In [172]:
average_f1

0.31780086589545975

#### Top 50 ICD codes identification

In [122]:
top_50_icd_codes = final_notes_with_icd_codes['icd_code'].value_counts().index[:50]
top_50_icd_codes

Index(['Y92.', 'O99.', 'I10.', 'E78.', 'Z87.', 'Z3A.', 'Z51.', 'Z85.', 'E11.',
       'Z79.', 'Z68.', 'F41.', 'K21.', 'F32.', 'Z66.', 'Z86.', 'E66.', 'I25.',
       'Z37.', 'E87.', 'J45.', 'I48.', 'E03.', 'F17.', 'G47.', 'Y83.', 'Z95.',
       'J96.', 'G93.', 'O26.', 'N17.', 'N18.', 'G89.', 'C78.', 'J44.', 'D64.',
       'R40.', 'O34.', 'Z80.', 'O09.', 'R11.', 'I50.', 'Z23.', 'S06.', 'M19.',
       'N40.', 'I12.', 'I95.', 'S62.', 'O70.'],
      dtype='object')

In [123]:
filtered_test_notes = final_notes_with_icd_codes[final_notes_with_icd_codes['icd_code'].isin(top_50_icd_codes)]

filtered_test_notes

,hadm_id,text,text_length,icd_code
3,20019282,\nName: ___ Unit No: ___\n...,2480,Y92.
7,20022343,\nName: ___ Unit No: _...,3001,O34.
9,20022343,\nName: ___ Unit No: _...,3001,Z3A.
10,20022343,\nName: ___ Unit No: _...,3001,Z37.
11,20025548,\nName: ___ Unit No: ___\n...,2961,O99.
...,...,...,...,...
8578,29990058,\nName: ___ Unit No: ___\n...,3051,Z3A.
8579,29990058,\nName: ___ Unit No: ___\n...,3051,Z37.
8581,29990058,\nName: ___ Unit No: ___\n...,3051,Z23.
8582,29990058,\nName: ___ Unit No: ___\n...,3051,Z86.


In [124]:
filtered_unique_hadm_notes = filtered_test_notes[['hadm_id', 'text']].drop_duplicates(subset='hadm_id')

In [125]:
filtered_unique_hadm_notes = filtered_unique_hadm_notes.reset_index(drop = True)
filtered_unique_hadm_notes

,hadm_id,text
0,20019282,\nName: ___ Unit No: ___\n...
1,20022343,\nName: ___ Unit No: _...
2,20025548,\nName: ___ Unit No: ___\n...
3,20027416,\nName: ___ Unit No: ___\n...
4,20028418,\nName: ___ Unit No: ___\n \...
...,...,...
929,29980814,\nName: ___ Unit No: _...
930,29980930,\nName: ___ Unit No: ___...
931,29987299,\nName: ___ Unit No: ___\...
932,29988390,\nName: ___ Unit No: ...


In [126]:
filtered_icd10_data = {
    'hadm_id': [],
    'icd10_codes': []
}

In [127]:
unique_icd_codes = set(filtered_test_notes['icd_code'])

In [128]:
# Step 3: Initialize TP, FP, FN, TN counts
filtered_counts = {icd_code: (0, 0, 0, 0) for icd_code in unique_icd_codes}



In [129]:
i = 0
for icd_code in unique_icd_codes:
    i+=1
    if i%10 == 0:
        print(f'{i} codes are done!')
    
    true_icd_codes = filtered_test_notes[filtered_test_notes['icd_code'] == icd_code]['hadm_id'].values
    predicted_icd_codes = result_df[result_df['icd_10_predicted'].apply(lambda x: icd_code in x)]['hadm_id'].values

    TP = np.intersect1d(true_icd_codes, predicted_icd_codes)
    FP = np.setdiff1d(predicted_icd_codes, true_icd_codes)
    FN = np.setdiff1d(true_icd_codes, predicted_icd_codes)
    TN = len(np.union1d(predicted_icd_codes, true_icd_codes)) -len(TP) -len(FP) -len(FN)


    filtered_counts[icd_code] = (len(TP), len(FP), len(FN), TN)

10 codes are done!
20 codes are done!
30 codes are done!
40 codes are done!
50 codes are done!


In [130]:
filtered_counts

{'Z66.': (4, 10, 96, 0),
 'E11.': (4, 0, 94, 0),
 'Z79.': (1, 5, 121, 0),
 'R11.': (15, 97, 26, 0),
 'F17.': (2, 1, 71, 0),
 'I48.': (18, 7, 56, 0),
 'O70.': (3, 11, 30, 0),
 'Z95.': (0, 0, 48, 0),
 'Z86.': (0, 0, 82, 0),
 'I95.': (14, 55, 20, 0),
 'E66.': (5, 7, 92, 0),
 'R40.': (1, 6, 19, 0),
 'E87.': (1, 4, 62, 0),
 'Y92.': (0, 2, 245, 0),
 'F41.': (1, 0, 105, 0),
 'O26.': (0, 0, 49, 0),
 'I25.': (6, 0, 61, 0),
 'C78.': (0, 0, 33, 0),
 'I10.': (0, 0, 224, 0),
 'G93.': (1, 1, 41, 0),
 'I50.': (7, 15, 32, 0),
 'O09.': (0, 3, 36, 0),
 'Z37.': (32, 7, 54, 0),
 'E78.': (8, 1, 189, 0),
 'F32.': (1, 1, 99, 0),
 'N18.': (11, 1, 41, 0),
 'G89.': (0, 3, 51, 0),
 'S62.': (4, 8, 13, 0),
 'K21.': (7, 2, 97, 0),
 'I12.': (0, 6, 34, 0),
 'Z85.': (2, 1, 109, 0),
 'S06.': (6, 8, 21, 0),
 'Z68.': (0, 1, 109, 0),
 'Z51.': (1, 11, 132, 0),
 'M19.': (0, 0, 31, 0),
 'Z23.': (1, 2, 39, 0),
 'Z3A.': (33, 21, 160, 0),
 'O99.': (0, 1, 130, 0),
 'O34.': (0, 1, 42, 0),
 'G47.': (6, 8, 65, 0),
 'E03.': (12, 4, 

In [131]:

# Calculate Precision and Recall for Each ICD-10 Code
filtered_precisions = {}
filtered_recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = filtered_counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    filtered_precisions[icd_code] = precision
    filtered_recalls[icd_code] = recall

In [132]:
filtered_precisions

{'Z66.': 0.2857142857142857,
 'E11.': 1.0,
 'Z79.': 0.16666666666666666,
 'R11.': 0.13392857142857142,
 'F17.': 0.6666666666666666,
 'I48.': 0.72,
 'O70.': 0.21428571428571427,
 'Z95.': 0,
 'Z86.': 0,
 'I95.': 0.2028985507246377,
 'E66.': 0.4166666666666667,
 'R40.': 0.14285714285714285,
 'E87.': 0.2,
 'Y92.': 0.0,
 'F41.': 1.0,
 'O26.': 0,
 'I25.': 1.0,
 'C78.': 0,
 'I10.': 0,
 'G93.': 0.5,
 'I50.': 0.3181818181818182,
 'O09.': 0.0,
 'Z37.': 0.8205128205128205,
 'E78.': 0.8888888888888888,
 'F32.': 0.5,
 'N18.': 0.9166666666666666,
 'G89.': 0.0,
 'S62.': 0.3333333333333333,
 'K21.': 0.7777777777777778,
 'I12.': 0.0,
 'Z85.': 0.6666666666666666,
 'S06.': 0.42857142857142855,
 'Z68.': 0.0,
 'Z51.': 0.08333333333333333,
 'M19.': 0,
 'Z23.': 0.3333333333333333,
 'Z3A.': 0.6111111111111112,
 'O99.': 0.0,
 'O34.': 0.0,
 'G47.': 0.42857142857142855,
 'E03.': 0.75,
 'N40.': 0.4,
 'J96.': 0.5,
 'N17.': 0.6842105263157895,
 'D64.': 0.0,
 'J44.': 0.5,
 'Z87.': 0,
 'Y83.': 0.07017543859649122,
 '

In [133]:
filtered_recalls

{'Z66.': 0.04,
 'E11.': 0.04081632653061224,
 'Z79.': 0.00819672131147541,
 'R11.': 0.36585365853658536,
 'F17.': 0.0273972602739726,
 'I48.': 0.24324324324324326,
 'O70.': 0.09090909090909091,
 'Z95.': 0.0,
 'Z86.': 0.0,
 'I95.': 0.4117647058823529,
 'E66.': 0.05154639175257732,
 'R40.': 0.05,
 'E87.': 0.015873015873015872,
 'Y92.': 0.0,
 'F41.': 0.009433962264150943,
 'O26.': 0.0,
 'I25.': 0.08955223880597014,
 'C78.': 0.0,
 'I10.': 0.0,
 'G93.': 0.023809523809523808,
 'I50.': 0.1794871794871795,
 'O09.': 0.0,
 'Z37.': 0.37209302325581395,
 'E78.': 0.04060913705583756,
 'F32.': 0.01,
 'N18.': 0.21153846153846154,
 'G89.': 0.0,
 'S62.': 0.23529411764705882,
 'K21.': 0.0673076923076923,
 'I12.': 0.0,
 'Z85.': 0.018018018018018018,
 'S06.': 0.2222222222222222,
 'Z68.': 0.0,
 'Z51.': 0.007518796992481203,
 'M19.': 0.0,
 'Z23.': 0.025,
 'Z3A.': 0.17098445595854922,
 'O99.': 0.0,
 'O34.': 0.0,
 'G47.': 0.08450704225352113,
 'E03.': 0.16216216216216217,
 'N40.': 0.058823529411764705,
 'J96.

In [134]:
# Step 6: Calculate Macro Precision and Macro Recall
macro_precision = sum(filtered_precisions.values()) / len(filtered_precisions)
macro_recall = sum(filtered_recalls.values()) / len(filtered_recalls)


In [135]:
macro_precision

0.36475883827588634

In [136]:
macro_recall

0.07810307463221956

In [140]:
# Calculate Micro Precision and Recall
total_TP = sum([filtered_counts[icd_code][0] for icd_code in unique_icd_codes])
total_FP = sum([filtered_counts[icd_code][1] for icd_code in unique_icd_codes])
total_FN = sum([filtered_counts[icd_code][2] for icd_code in unique_icd_codes])

# Handle division by zero
micro_precision = total_TP / (total_TP + total_FP) if (total_TP + total_FP) != 0 else 0
micro_recall = total_TP / (total_TP + total_FN) if (total_TP + total_FN) != 0 else 0

print("Micro Precision:", micro_precision)
print("Micro Recall:", micro_recall)

Micro Precision: 0.3860759493670886
Micro Recall: 0.062229023208365215


In [74]:
# Calculate F1 Score for Each ICD-10 Code
filtered_f1_scores = {}
for icd_code in unique_icd_codes:
    precision = filtered_precisions[icd_code]
    recall = filtered_recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    filtered_f1_scores[icd_code] = f1_score


In [75]:
# Step 8: Calculate Macro F1 Score
macro_f1_score = sum(filtered_f1_scores.values()) / len(filtered_f1_scores)

# Now you have macro precision, macro recall, and macro F1 score for your multi-label classification evaluation
macro_f1_score


0.10424185751195485

In [141]:
# Calculate Micro F1 Score
micro_precision = total_TP / (total_TP + total_FP) if (total_TP + total_FP) != 0 else 0
micro_recall = total_TP / (total_TP + total_FN) if (total_TP + total_FN) != 0 else 0

# Handle division by zero for precision and recall in Micro F1 Score
if micro_precision + micro_recall == 0:
    micro_f1_score = 0
else:
    micro_f1_score = 2 * (micro_precision * micro_recall) / (micro_precision + micro_recall)

print("Micro F1 Score:", micro_f1_score)

Micro F1 Score: 0.10718207775093345


In [76]:
# Calculate Precision and Recall for Each ICD-10 Code
filtered_precisions = {}
filtered_recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = filtered_counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    filtered_precisions[icd_code] = precision
    filtered_recalls[icd_code] = recall    
    
    
    
    
    
# Filter out zero precision values and calculate average precision
average_precision = np.mean([precision for precision in filtered_precisions.values() if precision > 0])

In [77]:
average_precision

0.5210840546798375

In [78]:
average_recall = np.mean([recall for recall in filtered_recalls.values() if recall > 0])

In [79]:
average_recall

0.11157582090317082

In [80]:
# Calculate F1 Score for Each ICD-10 Code
filtered_f1_scores = {}
for icd_code in unique_icd_codes:
    precision = filtered_precisions[icd_code]
    recall = filtered_recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    filtered_f1_scores[icd_code] = f1_score


In [81]:
average_f1 = np.mean([f1_score for f1_score in filtered_f1_scores.values() if f1_score > 0])

In [82]:
average_f1

0.1489169393027926